##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Authentication


In order to access send prompts to a tuned model, we must authenticate using OAuth

In [1]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  from google.colab import userdata
  import pathlib
  pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=540945254079-nisd41l30lgoh4i67jb298esmfku15p7.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=YmeNok7f2oAoAKIvCSjutwPG5dhbIv&access_type=offline&code_challenge=7f_R6BABDzg84Q3HLebcvYSyIPNs7KQw96761aexYeY&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=YmeNok7f2oAoAKIvCSjutwPG5dhbIv&code=4/0AdLIrYewbifmcz9M_qXCQKG925TYAYWZdvR6ssNuVsZgixDmV9-k385DerMQL0VK2LbgTQ&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generati

### Install & import

In [2]:
!pip install -U -q google-generativeai

In [3]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import io
import json
import mimetypes
import pathlib
import pprint
import requests

import PIL.Image
import IPython.display
from IPython.display import Markdown

## Parse the arguments



In [4]:
model = 'tunedModels/cpetshortviastudio02-nk5gwfiou8mo' # @param {isTemplate: true}

generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MC41LCJ0b3BfcCI6MSwidG9wX2siOjAsIm1heF9vdXRwdXRfdG9rZW5zIjo4MTkyLCJzdG9wX3NlcXVlbmNlcyI6W119' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
stream = False

In [191]:
print('Model: ', model)
print('Generation config:', json.dumps(generation_config, indent=2))
print('Safety settings: ', json.dumps(safety_settings, indent=2))

Model:  tunedModels/cpetshortviastudio02-nk5gwfiou8mo
Generation config: {
  "temperature": 0.5,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 8192,
  "stop_sequences": []
}
Safety settings:  [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]


In [192]:
# Import test dataset from local csv file
import pandas as pd
data = pd.read_csv("file://localhost/content/dialogsum.test.truncated.csv")
data = data.drop(columns=['topic1','summary2','topic2','summary3','topic3'])
data

,fname,dialogue,summary1
0,test_0,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...
1,test_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....
2,test_2,"#Person1#: Kate, you never believe what's happ...",#Person1# tells Kate that Masha and Hero get d...
3,test_3,"#Person1#: Happy Birthday, this is for you, Br...",#Person1# and Brian are at the birthday party ...
4,test_4,#Person1#: This Olympic park is so big!\n#Pers...,#Person1# is surprised at the Olympic Stadium'...
...,...,...,...
95,test_95,#Person1#: Thank you. Steven. That was the mos...,Steven and Lin just had a great meal. Then the...
96,test_96,"#Person1#: Bill, how can you hear so happy tod...",Bill is happy because he made a move to know h...
97,test_97,#Person1#: I'd like to pay my bill now. \n#Per...,#Person2# checks Tom Wilson's information and ...
98,test_98,#Person1#: Carol telephone.\n#Person2#: Who is...,Susan calls Carol to ask about the party time....


In [222]:
# format testing dataset

testing_data = {}
for _, row in data.iterrows():
  map = {}
  map['parts'] = [{'text': row['dialogue']}]
  testing_data[row['fname']] = map



## Call the API

In [200]:
# Call the model and store response for each item in the test data in array
gemini = genai.GenerativeModel(model_name=model)
responses = []
results = {}

for fname in testing_data:
  dialogue = testing_data[fname]['parts'][0]['text']
  response = gemini.generate_content(testing_data[fname],
                                     generation_config=generation_config,
                                     safety_settings=safety_settings,
                                     stream=False)
  responses.append(response)

  if response.candidates[0].finish_reason == 1:
    results[fname] = {'dialogue': dialogue, 'summary': response.text}
  else:
    results[fname] = {'dialogue': dialogue, 'summary': response.candidates[0].finish_reason }


In [224]:
# view all responses from test dataset
for response in responses:
  print(response.candidates)

[content {
  parts {
    text: "#Person1# dictates a memo to #Person2# to restrict the use of Instant Message programs during working hours. #Person2# asks questions about the memo and then continues with the dictation."
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]
[content {
  parts {
    text: "#Person2# got stuck in traffic again. #Person1# suggests #Person2# take public transport or bike to work. #Person2# agrees to quit driving to work."
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  proba

In [201]:
# Export results to local csv file

df = pd.DataFrame.from_dict(results, orient='index')
df.index.name = 'fname'
df = df.reset_index()
df
df.to_csv('dialogsum.test.truncated.output.csv', index=False)
